In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_community.llms import HuggingFaceHub
from crewai import LLM
import os

# Setting up the LLMs to use for our agents.
# We can also keep different LLMs for different agents

# llm = HuggingFaceHub(
#     repo_id = "mistralai/Mistral-7B-Instruct-v0.3",
#     huggingfacehub_api_token = os.getenv("HF_TOKEN"),
#     task = "text-generation",
# )

llm = LLM(model = "openai/gpt-4o-mini")

### Initialize the Agents

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
sales_rep_agent = Agent(
    role = "Sales Representative",
    goal = "Identify high-value leads that match our ideal customer profile",
    backstory = """As a part of the dynamic sales team at CrewAI, your mission is to scour the digital landscape for potential leads. 
    Armed with cutting-edge tools and a strategic mindset, you analyze data, trends, and interactions to unearth opportunities that others might overlook. 
    Your work is crucial in paving the way for meaningful engagements and driving the company's growth.""",
    allow_delegation = False,
    verbose = True
)

In [ ]:
lead_sales_rep_agent = Agent(
    role = "Lead Sales Representative",
    goal = "Nurture leads with personalized, compelling communications",
    backstory = """Within the vibrant ecosystem of CrewAI's sales department, you stand out as the bridge between potential clients and the solutions they need. 
    By creating engaging, personalized messages, you not only inform leads about our offerings but also make them feel seen and heard. 
    Your role is pivotal in converting interest into action, guiding leads through the journey from curiosity to commitment.""",
    allow_delegation = False,
    verbose = True
)

### Creating Tools

In [ ]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

In [ ]:
directory_read_tool = DirectoryReadTool(directory = './instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

Create a custom tool using crewAi's BaseTool class

In [ ]:
from crewai_tools import BaseTool

In [ ]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = "Analyzes the sentiment of text to ensure positive and engaging communication."
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [ ]:
sentiment_analysis_tool = SentimentAnalysisTool()

### Initialize the Tasks

In this part we initialize the tasks for each agent. The Task describes what you expect the agent to do and how should the output be produced.
A well defined task should have a clear description & expectation and good context.

In [ ]:
lead_profiling_task = Task(
    description = """Conduct an in-depth analysis of {lead_name}, a company in the {industry} sector that recently showed interest in our solutions. 
    Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments,
    and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively. 
    Don't make assumptions and only use information you absolutely sure about.""",
    expected_output = """A comprehensive report on {lead_name}, including company background, key personnel, recent milestones, and identified needs. 
    Highlight potential areas where our solutions can provide value, and suggest personalized engagement strategies.""",
    tools = [directory_read_tool, file_read_tool, search_tool],
    agent = sales_rep_agent,
)

In [ ]:
personalized_outreach_task = Task(
    description = """Using the insights gathered from the lead profiling report on {lead_name}, craft a personalized outreach campaign 
    aimed at {key_decision_maker}, the {position} of {lead_name}. The campaign should address their recent {milestone} 
    and how our solutions can support their goals. Your communication must resonate with {lead_name}'s company culture and values, 
    demonstrating a deep understanding of their business and needs. 
    Don't make assumptions and only use information you absolutely sure about.""",
    expected_output = 
        """A series of personalized email drafts tailored to {lead_name}, specifically targeting {key_decision_maker}. 
        Each draft should include a compelling narrative that connects our solutions with their recent achievements and future goals. 
        Ensure the tone is engaging, professional, and aligned with {lead_name}'s corporate identity.""",
    tools = [sentiment_analysis_tool, search_tool],
    agent = lead_sales_rep_agent,
)

### Create the Crew
Crew assembles the agents and their tasks togethers to create a flow for execution. Order of the task is important.

In [ ]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)